In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 11040



### retain and sum

In [2]:
sas.submitLST(f"""
    data work.test;
        set sashelp.class;
        length heaviest_names $100.;
        retain total_weight 0 max_weight 0 heaviest_names "";
              
        total_weight = sum(total_weight, weight);
        max_weight = max(max_weight, weight);
        if weight = max_weight then heaviest_names = catx(', ', heaviest_names, name);
              
        total_weight2 + weight;
    run;
    proc print data=work.test;
        var name weight max_weight heaviest_names total_weight total_weight2;
    run;
""")

Obs,Name,Weight,max_weight,heaviest_names,total_weight,total_weight2
1,Alfred,112.5,112.5,Alfred,112.5,112.5
2,Alice,84.0,112.5,Alfred,196.5,196.5
3,Barbara,98.0,112.5,Alfred,294.5,294.5
4,Carol,102.5,112.5,Alfred,397.0,397.0
5,Henry,102.5,112.5,Alfred,499.5,499.5
6,James,83.0,112.5,Alfred,582.5,582.5
7,Jane,84.5,112.5,Alfred,667.0,667.0
8,Janet,112.5,112.5,"Alfred, Janet",779.5,779.5
9,Jeffrey,84.0,112.5,"Alfred, Janet",863.5,863.5
10,John,99.5,112.5,"Alfred, Janet",963.0,963.0
